# 任务4 - 模型评估（2天）

- 记录5个模型（逻辑回归、SVM、决策树、随机森林、XGBoost）
- 关于accuracy、precision，recall和F1-score、auc值的评分表格

In [9]:
import numpy as np
import pandas as pd
data=pd.read_csv(r'data.csv',encoding='gbk')


In [10]:
# #分割数据集为训练集和测试集
# from sklearn.model_selection import train_test_split
# Y=data['status']
# X=data.drop(['status'],axis=1)
# X_train,X_test,y_train,y_test = train_test_split(X,Y,test_size=0.3,random_state=2018)


In [19]:
# #决策树算法和随机森林算法：
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.ensemble import RandomForestClassifier

# #导入模型评估分数， roc_auc_score--auc值  
# from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,roc_auc_score,roc_curve
# import matplotlib.pyplot as plt
# import seaborn as sns

# model_list = [('dt',DecisionTreeClassifier()),('rf',RandomForestClassifier())]
# score_list = [('准确率',accuracy_score),('精准率',precision_score),('召回率',recall_score),('F1-score',f1_score),('AUC值',roc_auc_score),('ROC',roc_curve)]

# 导入库
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

from sklearn.model_selection import RandomizedSearchCV,cross_val_predict
from scipy.stats import uniform
from sklearn import metrics
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,roc_auc_score,roc_curve
import matplotlib.pyplot as plt


# 获取进行特征选择后的数据集

x = data.iloc[:,:-1]
y = data.iloc[:,-1]

In [20]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.3,random_state=2018)

# 对训练数据集进行标准化处理
scaler = StandardScaler()
x_train_standard = scaler.fit_transform(x_train)
x_test_standard = scaler.fit_transform(x_test)


ValueError: could not convert string to float: '2018/4/7'

In [ ]:
def get_scores(label, y_train, y_test, y_train_predict, y_test_predict, y_train_proba, y_test_proba):
    train_accuracy = metrics.accuracy_score(y_train, y_train_predict)
    test_accuracy = metrics.accuracy_score(y_test, y_test_predict)
    # 精准率
    train_precision = metrics.precision_score(y_train, y_train_predict)
    test_precision = metrics.precision_score(y_test, y_test_predict)
    # 召回率
    train_recall = metrics.recall_score(y_train, y_train_predict)
    test_recall = metrics.recall_score(y_test, y_test_predict)
    # F1-score
    train_f1_score = metrics.f1_score(y_train, y_train_predict)
    test_f1_score = metrics.f1_score(y_test, y_test_predict)
    # AUC
    train_auc = metrics.roc_auc_score(y_train, y_train_proba)
    test_auc = metrics.roc_auc_score(y_test, y_test_proba)
    # ROC
    train_fprs, train_tprs, train_thresholds = metrics.roc_curve(y_train, y_train_proba)
    test_fprs, test_tprs, test_thresholds = metrics.roc_curve(y_test, y_test_proba)
    
    plt.plot(train_fprs, train_tprs, label=label+' train ROC', linewidth=2)
    plt.plot(test_fprs, test_tprs, label=label+' test ROC', linewidth=2)
    plt.title("ROC Curve")
    plt.xlabel("FPR")
    plt.ylabel("TPR")
    plt.legend()
    plt.show()
    # 输出评分
    print("训练集准确率：", train_accuracy)
    print("测试集准确率：", test_accuracy)
    print("训练集精准率：", train_precision)
    print("测试集精准率：", test_precision)
    print("训练集召回率：", train_recall)
    print("测试集召回率：", test_recall)
    print("训练集F1-score：", train_f1_score)
    print("测试集F1-score：", test_f1_score)
    print("训练集AUC：", train_auc)
    print("测试集AUC：", test_auc)
    
    train = [train_accuracy, train_precision, train_recall, train_f1_score, train_auc]
    test = [test_accuracy, test_precision, test_recall, test_f1_score, test_auc]
    return train, test


In [ ]:
#SVM

svm_linearSVC = LinearSVC(random_state=2018)
svm_linearSVC.fit(x_train_standard, y_train)
y_train_predict = svm_linearSVC.predict(x_train_standard)
y_test_predict = svm_linearSVC.predict(x_test_standard)
y_train_proba = svm_linearSVC.decision_function(x_train_standard)
y_test_proba = svm_linearSVC.decision_function(x_test_standard)
SVM_train, SVM_test = get_scores('SVM', y_train, y_test, y_train_predict, y_test_predict, y_train_proba, y_test_proba)


In [ ]:
# 决策树
tree = DecisionTreeClassifier(random_state=2018)
tree.fit(x_train_standard, y_train)
y_train_predict = tree.predict(x_train_standard)
y_test_predict = tree.predict(x_test_standard)
y_train_proba = tree.predict_proba(x_train_standard)[:, 1]
y_test_proba = tree.predict_proba(x_test_standard)[:, 1]
DT_train, DT_test = get_scores('DT', y_train, y_test, y_train_predict, y_test_predict, y_train_proba, y_test_proba)



In [ ]:
# 随机森林
rf = RandomForestClassifier(n_estimators=1000,criterion='gini',oob_score=True,
                               random_state=2018,verbose=0,n_jobs=-1)
rf.fit(x_train_standard, y_train)
y_train_predict = rf.predict(x_train_standard)
y_test_predict = rf.predict(x_test_standard)
y_train_proba = rf.predict_proba(x_train_standard)[:, 1]
y_test_proba = rf.predict_proba(x_test_standard)[:, 1]
RM_train, RM_test = get_scores('RM', y_train, y_test, y_train_predict, y_test_predict, y_train_proba, y_test_proba)


In [ ]:
# XGboost
xgb = XGBClassifier(random_state=2018)
xgb.fit(x_train_standard, y_train)
y_train_predict = xgb.predict(x_train_standard)
y_test_predict = xgb.predict(x_test_standard)
y_train_proba = xgb.predict_proba(x_train_standard)[:, 1]
y_test_proba = xgb.predict_proba(x_test_standard)[:, 1]
XGBoost_train, XGBoost_test = get_scores('XGBoost', y_train, y_test, y_train_predict, y_test_predict, y_train_proba, y_test_proba)
